In [21]:
import pandas as pd
import random
import time
import numpy as np


from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold



from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")

In [7]:
B = pd.read_pickle("~/bge-analysis-simv3/B.pkl")
A = pd.read_pickle("~/bge-analysis-simv3/A.pkl")

tableg=A.T
species_onesall = [] 
for species in range(16):
    species_onesall.append(tableg[tableg[species]==1.0][species].index.to_list())
species_zerosall = []
for species in range(16):
    species_zerosall.append(tableg[tableg[species]==0.0][species].index.to_list())

In [8]:
pa_redv1 = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/ophelli-ryan/eff-16-list1-red-pa.pkl")

##### compile the well numbers for each species for its presence 

species_ones = []
for i in range(16):
    species_ones.append(pa_redv1[pa_redv1[i]==1.0]["well number"].values)

ones_len = [] 
for ii in range(len(species_ones)):
    ones_len.append(len(species_ones[ii])) 

In [9]:
steadystate = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/ophelli-ryan/eff-16-list1-red-sst-REL.pkl")
sst = steadystate.T

In [11]:
cv_splitsl = [3,5,7] 
max_depthl = [2,5,10,15,20]
ntrees = [100]
#max_featuresl = ["auto","sqrt","log2"]
max_featuresl = [0.33,0.5]

In [12]:
listll = [7,15] 

df = [] 
for kk in listll:
    df.append(pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-v2/2pow15-matrices/16l1-"+str(kk)+"-bge.pkl"))

In [15]:
df2 = pd.concat([df[0],df[1]])
#df2 = pd.concat([df[0],df[1],df[2],df[3],df[4],df[5],df[6]])

dflin = [] 
for kk in listll:
    dflin.append(pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-v2/2pow15-matrices/16l1-"+str(kk)+"-lin.pkl"))

dflin2 = pd.concat([dflin[0],dflin[1]])
#dflin2  = pd.concat([dflin[0],dflin[1],dflin[2],dflin[3],dflin[4],dflin[5],dflin[6]])

In [16]:
bias_var_listm = [3,3,3,3,3,3,3,3,5,3,5,3,3,7,5,7]

In [17]:
startt = time.time()
rfper = [] 
for iteras in range(5):
    runidx = df2[df2[0]==iteras]
    for species in range(16):
        speciesc = species 
        subsetall = A[species_onesall[species]].T
        subsetall["indexinwhole"] = subsetall.index
        subsetall.index = np.arange(0,2**15,1)
        givendata = subsetall[subsetall["indexinwhole"].isin(species_ones[species])]
        sstgiven = steadystate.T[subsetall[subsetall["indexinwhole"].isin(species_ones[species])]["indexinwhole"].values].T
        sstgiven.index = givendata.index
        m = bias_var_listm[species]
        spidx = runidx[runidx[2]==species][:m]
        for folds in range(m):
            trfold = spidx[5].iloc[folds]
            tefold = spidx[4].iloc[folds]
            sstdatatrY = sstgiven.T[trfold][:16].T[species]
            trX = givendata.T[:16][trfold].T
            sstact_tey = sstgiven.T[tefold][:16].T[species]
            teX =  givendata.T[:16][tefold].T
            for md in max_depthl:
                for n_est in ntrees:
                    for mfeatures in max_featuresl:
                            regr = RandomForestRegressor(max_depth=md,n_estimators=n_est,max_features=mfeatures)
                            regr.fit(trX,sstdatatrY)
                            ypred = regr.predict(teX)
                            rfper.append([species,iteras,m,folds,md,n_est,mfeatures,ypred,sstact_tey.values,trfold,tefold])
print(time.time() - startt)            

332.5965025424957


In [18]:
rdf = pd.DataFrame(rfper,columns = ["species","run idx","k-fold","k-fold idx","max depth","trees","feats","ypred","yact","train data","te data"])

In [19]:
rdf.to_pickle("RF-16sp-OV-REL-5runs-featsarefrac.pkl")

In [20]:
rdf

,species,run idx,k-fold,k-fold idx,max depth,trees,feats,ypred,yact,train data,te data
0,0,0,3,0,2,100,0.33,"[0.5875578557735204, 0.34797326000687157, 0.54...","[0.3391491238628783, 0.16302582467502283, 0.18...","[0, 2, 4, 20, 32, 128, 132, 4102, 4230, 8199, ...","[1, 260, 4226, 4228, 4256, 8197, 24581]"
1,0,0,3,0,2,100,0.50,"[0.7228678964623014, 0.32205782861737764, 0.59...","[0.3391491238628783, 0.16302582467502283, 0.18...","[0, 2, 4, 20, 32, 128, 132, 4102, 4230, 8199, ...","[1, 260, 4226, 4228, 4256, 8197, 24581]"
2,0,0,3,0,5,100,0.33,"[0.7131989137758097, 0.4100371921857258, 0.473...","[0.3391491238628783, 0.16302582467502283, 0.18...","[0, 2, 4, 20, 32, 128, 132, 4102, 4230, 8199, ...","[1, 260, 4226, 4228, 4256, 8197, 24581]"
3,0,0,3,0,5,100,0.50,"[0.7622055523487399, 0.3840877061639671, 0.627...","[0.3391491238628783, 0.16302582467502283, 0.18...","[0, 2, 4, 20, 32, 128, 132, 4102, 4230, 8199, ...","[1, 260, 4226, 4228, 4256, 8197, 24581]"
4,0,0,3,0,10,100,0.33,"[0.7182419902155617, 0.40089846653743, 0.46767...","[0.3391491238628783, 0.16302582467502283, 0.18...","[0, 2, 4, 20, 32, 128, 132, 4102, 4230, 8199, ...","[1, 260, 4226, 4228, 4256, 8197, 24581]"
...,...,...,...,...,...,...,...,...,...,...,...
3095,15,4,7,6,10,100,0.50,"[0.7844164712885376, 0.44048849562231107, 0.92...","[0.767183023378863, 0.44475043202614783, 0.983...","[0, 1, 2, 3, 4, 7, 8, 15, 16, 17, 18, 22, 23, ...","[14, 19, 67, 144, 274, 1232, 2114, 2115, 8272,..."
3096,15,4,7,6,15,100,0.33,"[0.7812192862557298, 0.43263786481424565, 0.86...","[0.767183023378863, 0.44475043202614783, 0.983...","[0, 1, 2, 3, 4, 7, 8, 15, 16, 17, 18, 22, 23, ...","[14, 19, 67, 144, 274, 1232, 2114, 2115, 8272,..."
3097,15,4,7,6,15,100,0.50,"[0.7531181656138339, 0.4226301871927689, 0.930...","[0.767183023378863, 0.44475043202614783, 0.983...","[0, 1, 2, 3, 4, 7, 8, 15, 16, 17, 18, 22, 23, ...","[14, 19, 67, 144, 274, 1232, 2114, 2115, 8272,..."
3098,15,4,7,6,20,100,0.33,"[0.7849726268123868, 0.4192057781162998, 0.904...","[0.767183023378863, 0.44475043202614783, 0.983...","[0, 1, 2, 3, 4, 7, 8, 15, 16, 17, 18, 22, 23, ...","[14, 19, 67, 144, 274, 1232, 2114, 2115, 8272,..."


In [ ]:
|